In [ ]:
%matplotlib inline
from rswarp.utilities.file_utils import *
from rswarp.utilities.beam_analysis import *
from matplotlib import pyplot as plt
import numpy as np
import h5py
import os

In [ ]:
#Import phase space data from all steps. Uses readparticles()
stepArrays = loadparticlefiles('diags/xySlice/hdf5/')
steps = stepArrays.keys()
steps.sort()

In [ ]:
eleccount = [len(stepArrays[s]['e-']) for s in steps]
emiteleccount = [len(stepArrays[s]['emitted e-']) for s in steps]
h2pcount = [len(stepArrays[s]['H2+']) for s in steps]
t = [stepArrays[s]['time'] for s in steps]

fig = plt.figure(figsize=(12,6))
plt.suptitle('Particle counts over time',fontsize=20)

plt.subplot(121)
plt.plot(t,eleccount,'g-', label='beam e-')
plt.xlabel('time (s)')
plt.legend(loc='lower right')

plt.subplot(122)
plt.plot(t,emiteleccount,'ro', label='emitted e-')
plt.plot(t,h2pcount,'b--', label='emitted H2+')
plt.xlabel('time (s)')
plt.legend(loc='lower right')


plt.show()

In [ ]:
# filteredsteps = steps[::]
filteredsteps = steps[-2:-1]
# filteredsteps = steps[0:1]
elecdata = [convertunits(stepArrays[s]['e-']) for s in filteredsteps]
emitelecdata = [convertunits(stepArrays[s]['emitted e-']) for s in filteredsteps]
h2pdata = [convertunits(stepArrays[s]['H2+']) for s in filteredsteps]
times = [stepArrays[s]['time'] for s in filteredsteps]
print("Length of data: %i steps, from %.2E to %.2E s" % (len(times),times[0],times[-1]))

In [ ]:
figs = []
def plotDecorate(fig,name):
    fig.suptitle("%s phase space for step %i time %.3E s" % (name,filteredsteps[i],times[i]))
    fig.axes[4].set_xlim(0.0,1.0) # z limits
    
def emitplotDecorate(fig,name):
    fig.suptitle("%s phase space for step %i time %.3E s" % (name,filteredsteps[i],times[i]))
    fig.axes[0].set_ylim(-1000,1000)
    fig.axes[1].set_ylim(-1000,1000)
    fig.axes[4].set_xlim(0.0,1.0) # z limits

for i in range(0,len(filteredsteps)):
    for d in [
        {'data': elecdata[i], 'color': 'g', 'name': 'e-', 'fraction': 0.33, 'plotDecorate': plotDecorate},
        {'data': emitelecdata[i], 'color': 'r', 'name': 'emitted e-', 'fraction': 1.0, 'plotDecorate': emitplotDecorate},
        {'data': h2pdata[i], 'color': 'b', 'name': 'H2+', 'fraction': 1.0, 'plotDecorate': emitplotDecorate},
    ]:
        data = d['data']
        if len(data) > 0:
            plotDecorate = d['plotDecorate']
            fraction = d['fraction']
            mask = np.random.choice([False, True], len(data), p=[1 - fraction, fraction])
            # print("Showing %i particles out of %i for case %s" % (sum(mask),len(data),d['name']))
            figs.append(plotphasespace(data[mask,:], color=d['color'], decorator=lambda fig: plotDecorate(fig,d['name'])))

In [ ]:
import opmd_viewer
from matplotlib import pyplot as plt
from opmd_viewer import OpenPMDTimeSeries
ts = OpenPMDTimeSeries('./diags/fields/electric')
print(opmd_viewer.openpmd_timeseries.data_reader.params_reader.read_openPMD_params(ts.h5_files[0]))
%matplotlib inline
if ts.avail_circ_modes is None:
    ts.avail_circ_modes = []
ts.slider()